# Setup and init all the values

In [1]:
import math
import random
import time
import numpy as np
import tensorflow as tf
import input_data

batch_size = 128
image_len = 25
flat_image_len = image_len * image_len

# X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = input_data.read_data_sets("./data", one_hot=False)

# Define all the functions that will help us

In [2]:
def create_pairs(x, y):
    '''
    Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []

    unique = np.unique(y)
    digit_indices = [np.where(y == i)[0] for i in unique]
    # Looks like [array([7223]) 2.0]
    negatives = np.array(filter(lambda x: len(x) == 1, digit_indices))
    # Looks like [array([   33,  6178,  8623,  9490, 10098]) 1.0]
    positives = np.array(filter(lambda x: len(x) >= 2, digit_indices))
    for d in positives:
        for i in range(len(d) / 2):
            z1, z2 = d[i], d[i + 1]
            pairs += [[x[z1], x[z2]]]

            z1, z2 = random.choice(negatives)[0], random.choice(negatives)[0]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def mlp(input_, input_dim, output_dim, name="mlp"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [input_dim, output_dim], tf.float32,
                            tf.random_normal_initializer(mean=0.001, stddev=0.02))
        return tf.nn.relu(tf.matmul(input_, w))


def mlpnet(image, _dropout):
    l1 = mlp(image, flat_image_len, batch_size, name='l1')
    l1 = tf.nn.dropout(l1, _dropout)
    l2 = mlp(l1, batch_size, batch_size, name='l2')
    l2 = tf.nn.dropout(l2, _dropout)

    l3 = mlp(l2, batch_size, batch_size, name='l3')
    l3 = tf.nn.dropout(l3, _dropout)
    l4 = mlp(l3, batch_size, batch_size, name='l4')
    l4 = tf.nn.dropout(l4, _dropout)

    l5 = mlp(l4, batch_size, batch_size, name='l5')
    return l5


def contrastive_loss(y, d):
    tmp = y * tf.square(d)
    tmp2 = (1 - y) * tf.square(tf.maximum((1 - d), 0))
    return tf.reduce_sum(tmp + tmp2) / batch_size / 2


def compute_accuracy(prediction, labels):
    return labels[prediction.ravel() < 0.5].mean()


def next_batch(s, e, inputs, labels):
    sample = np.random.randint(len(inputs), size=batch_size)
    input1 = inputs[sample, 0]
    input2 = inputs[sample, 1]
    y = np.reshape(labels[sample], (batch_size, 1))
    return input1, input2, y


# Create positive / negative pairs, models and loss

In [ ]:
# Initializing the variables
print('Initializing the variables')
init = tf.initialize_all_variables()
# the data, shuffled and split between train and test sets

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 10, 0.1, staircase=True)

# create training+test positive and negative pairs
tr_pairs, tr_y = create_pairs(X_train, y_train)
te_pairs, te_y = create_pairs(X_test, y_test)

images_L = tf.placeholder(tf.float32, shape=([None, flat_image_len]), name='L')
images_R = tf.placeholder(tf.float32, shape=([None, flat_image_len]), name='R')
labels = tf.placeholder(tf.float32, shape=([None, 1]), name='gt')
dropout_f = tf.placeholder("float")

print('Initializing the models')
with tf.variable_scope("siamese") as scope:
    model1 = mlpnet(images_L, dropout_f)
    scope.reuse_variables()
    model2 = mlpnet(images_R, dropout_f)

distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.sub(model1, model2), 2), 1, keep_dims=True))
loss = contrastive_loss(labels, distance)

# contrastice loss
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'l' in var.name]
batch = tf.Variable(0)
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

# Launch the graph

In [3]:
print('Launch the graph')
with tf.Session() as sess:
    summary_writer = tf.train.SummaryWriter('./logs', sess.graph)

    # sess.run(init)
    tf.initialize_all_variables().run()
    # Training cycle
    for epoch in range(30):
        avg_loss = 0.
        avg_acc = 0.
        total_batch = int(X_train.shape[0] / batch_size)
        start_time = time.time()
        # Loop over all batches
        for i in range(total_batch):
            s = i * batch_size
            e = (i + 1) * batch_size
            # Fit training using batch data
            input1, input2, y = next_batch(s, e, tr_pairs, tr_y)
            _, loss_value, predict = sess.run([optimizer, loss, distance],
                                              feed_dict={images_L: input1, images_R: input2, labels: y, dropout_f: 0.9})
            feature1 = model1.eval(feed_dict={images_L: input1, dropout_f: 0.9})
            feature2 = model2.eval(feed_dict={images_R: input2, dropout_f: 0.9})
            tr_acc = compute_accuracy(predict, y)
            if math.isnan(tr_acc) and epoch != 0:
                pass
            avg_loss += loss_value
            avg_acc += tr_acc * 100
        duration = time.time() - start_time
        print('epoch %d  time: %f loss %0.5f acc %0.2f' % (
                epoch, duration, avg_loss / (total_batch), avg_acc / total_batch))
    y = np.reshape(tr_y, (tr_y.shape[0], 1))
    predict = distance.eval(feed_dict={images_L: tr_pairs[:, 0], images_R: tr_pairs[:, 1], labels: y, dropout_f: 1.0})
    tr_acc = compute_accuracy(predict, y)
    print('Accuract training set %0.2f' % (100 * tr_acc))

    # Test model
    predict = distance.eval(feed_dict={images_L: te_pairs[:, 0], images_R: te_pairs[:, 1], labels: y, dropout_f: 1.0})
    y = np.reshape(te_y, (te_y.shape[0], 1))
    te_acc = compute_accuracy(predict, y)
    print('Accuract test set %0.2f' % (100 * te_acc))


Initializing the variables
Initializing the models
Launch the graph


/users/ugrad/h/haustind/anaconda2/envs/tensorflow/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/users/ugrad/h/haustind/anaconda2/envs/tensorflow/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


epoch 0  time: 0.896392 loss 0.13688 acc nan
epoch 1  time: 0.876787 loss 0.12532 acc 54.60
epoch 2  time: 0.900473 loss 0.12079 acc 56.42
epoch 3  time: 1.114103 loss 0.11712 acc 59.16
epoch 4  time: 0.885876 loss 0.11100 acc 62.35
epoch 5  time: 0.893907 loss 0.10388 acc 65.41
epoch 6  time: 0.879561 loss 0.10083 acc 66.60
epoch 7  time: 0.891259 loss 0.09852 acc 67.64
epoch 8  time: 1.009935 loss 0.09391 acc 69.88
epoch 9  time: 0.921697 loss 0.09142 acc 70.46
epoch 10  time: 0.879820 loss 0.09092 acc 70.98
epoch 11  time: 0.973777 loss 0.08872 acc 71.49
epoch 12  time: 0.882592 loss 0.08444 acc 74.06
epoch 13  time: 0.898443 loss 0.07820 acc 76.06
epoch 14  time: 0.985998 loss 0.07739 acc 76.79
epoch 15  time: 0.969723 loss 0.07316 acc 77.57
epoch 16  time: 0.979054 loss 0.07236 acc 78.41
epoch 17  time: 0.932434 loss 0.06820 acc 80.17
epoch 18  time: 0.977523 loss 0.06523 acc 81.42
epoch 19  time: 0.936731 loss 0.06394 acc 81.74
epoch 20  time: 0.975970 loss 0.06284 acc 82.29
epoc

# A tensorboard screenshot showing that the architecture is, indeed, a siamese architecture

In [4]:
import requests
from StringIO import StringIO
from IPython.display import Image, display

url = "https://s13.postimg.org/i75d2auh3/graph_run.png"
Image(url= url)

**Description of train/test split**

I did a 80/20 train/test split using `train_test_split(X, y, test_size=0.2)`. To get the +/- test split I divided the images into 2 groups where one contained only people who had a single image (SIG), the other containing images with multiple images per person(MIG). For each person in MIG I would add an equal number of pics from SIG to create a balanced +/- training set. 


**Description of resnet architecture** 

I used a 5 layer network with relus for my nonlinearities, and some dropout between each layer.


**Assessed whether or not architecture was working**

I used a distance scoring function along with a check of how many images where correctly classified.


**The final performance of classifier**

> epoch 29  time: 0.871985 loss 0.04132 acc 89.19

>Accuract training set 92.03

>Accuract test set 64.11